In [1]:
# import required libraries
import pytz
import os
import numpy as np
#import warnings.filterwarnings('ignore')
import dotenv
import asyncpg
import asyncio
import pandas as pd 
from datetime import datetime

# load environment variables
dotenv.load_dotenv()

# database url
DATABASE_URL = f"postgresql://{os.getenv('user')}:{os.getenv('password')}@{os.getenv('host')}:{os.getenv('port')}/{os.getenv('db')}"

# create an async database connection pool
async def connect_db():
    """Initialize the async database connection pool."""
    global db_pool
    db_pool = await asyncpg.create_pool(DATABASE_URL, min_size=1, max_size=10)
    print('Database connected successfully!')

# close the connection pool
async def disconnect_db():
    """Close the database connection pool."""
    global db_pool
    if db_pool:
        await db_pool.close()
    print('Database connection closed!')


async def table2df(schema_name:str,table_name: str, timeframe:str):
    """
        Get data from a table in the database and return as a Pandas DataFrame.

        parameters
        ----------
        table_name (str): The name of the table in the database.

        returns
        -------
        df (pd.DataFrame): The data from the table as a Pandas DataFrame.
    """

    try:
        # get a connection from the pool
        async with db_pool.acquire() as conn:
            
            # query (utc)
            query = f"SELECT * FROM {schema_name}.{table_name}"
            rows = await conn.fetch(query)

            # convert asyncpg.Record to a list of dictionaries & convert to pandas
            df = pd.DataFrame([dict(row) for row in rows])

            if df.empty:
                return df  # return an empty DataFrame if no data

            # standardize all column names to lowercase
            df.columns = df.columns.str.lower()

            # set date_time column as index
            df.set_index(f"bucket_{timeframe}", inplace=True)

            # ensure the date_time column is in datetime format (optional, but recommended)
            df.index = pd.to_datetime(df.index)

            df.replace(0.0, pd.NA, inplace=True)
            df.dropna(how="any", inplace=True)
            # ensure numeric columns are float64
            df = df.astype("float")

            return df

    except Exception as e:
        print("An error occurred:", e)
        return pd.DataFrame()  # return an empty DataFrame in case of failure


In [2]:
await connect_db()

Database connected successfully!


In [3]:
# retrieve 6hour gold data
gold_6h = await table2df("gold", "ohlc_data_6hr_bid_xau_usd", "6h")
# retrieve daily gold data 
gold_daily = await table2df("gold", "ohlc_data_daily_bid_xau_usd", "daily")

# retrieve 6hour silver data
silver_6h = await table2df("silver", "ohlc_data_6hr_bid_xag_usd", "6h")
# retrieve daily silver data 
silver_daily = await table2df("silver", "ohlc_data_daily_bid_xag_usd", "daily")

# retrieve 6hour platinum data
platinum_6h = await table2df("platinum", "ohlc_data_6hr_bid_xpt_usd", "6h")
# retrieve daily platinum data
platinum_daily = await table2df("platinum", "ohlc_data_daily_bid_xpt_usd", "daily")

# retrieve 6hour sgd data
sgd_6h = await table2df("sgd", "ohlc_data_6hr_bid_usd_sgd", "6h")
# retrieve daily sgd data
sgd_daily = await table2df("sgd", "ohlc_data_daily_bid_usd_sgd", "daily")

# retrieve 6hour myr data
myr_6h = await table2df("myr", "ohlc_data_6hr_bid_usd_myr", "6h")
# retrieve daily myr data
myr_daily = await table2df("myr", "ohlc_data_daily_bid_usd_myr", "daily")

In [4]:
print(gold_6h.tail(3))
print(silver_6h.tail(3))

                              open     high      low    close    volume
bucket_6h                                                              
2025-08-11 18:00:00+00:00  3357.55  3358.63  3341.75  3349.06   81956.0
2025-08-12 00:00:00+00:00  3348.71  3357.46  3346.08  3354.51  111059.0
2025-08-12 06:00:00+00:00  3354.51  3354.82  3342.14  3344.60   14021.0
                             open    high     low   close   volume
bucket_6h                                                         
2025-08-11 18:00:00+00:00  37.776  37.798  37.495  37.679  17463.0
2025-08-12 00:00:00+00:00  37.673  37.950  37.671  37.943  34309.0
2025-08-12 06:00:00+00:00  37.944  37.950  37.838  37.874   4550.0


In [5]:
print(gold_daily.head(3))
print(gold_daily.tail(3))

                              open     high      low    close    volume
bucket_daily                                                           
2025-06-05 00:00:00+00:00  3377.40  3403.42  3339.34  3361.95  407944.0
2025-06-06 00:00:00+00:00  3361.79  3375.53  3307.06  3309.85  370173.0
2025-06-08 00:00:00+00:00  3312.09  3321.08  3308.98  3318.02   23043.0
                              open     high      low    close    volume
bucket_daily                                                           
2025-08-10 00:00:00+00:00  3396.69  3404.08  3389.87  3391.48   25873.0
2025-08-11 00:00:00+00:00  3391.29  3391.58  3341.28  3349.06  528817.0
2025-08-12 00:00:00+00:00  3348.71  3357.46  3342.14  3344.60  125080.0


In [6]:
def calculate_slowD(df: pd.DataFrame, k_period: int = 9, d_period: int = 3) -> pd.DataFrame:
    """A stochastic function that calculates the Fast %K & Slow %D using EMA.
    
    Parameters
    ----------
    df: pd.DataFrame (Input dataframe containing OHLC data.)
    k_period: int, optional (Period to calculate the Fast %K <default is 9>.)
    d_period: int, optional (Period to calculate the Slow %D <default is 3>.)
    
    Returns
    -------
    pd.DataFrame (DataFrame that contains Fast %K, Fast %D (EMA), and Slow %D (EMA).)
    """

    # find the highest high market price in the k period
    df['highest_high'] = df['high'].rolling(window=k_period).max()

    # find the lowest low market price in the k period
    df['lowest_low'] = df['low'].rolling(window=k_period).min()

    # calculate Fast %K
    df['fastk'] = ((df['close'] - df['lowest_low']) / (df['highest_high'] - df['lowest_low'])) * 100

    # calculate Fast %D (EMA of Fast %K with period 1, which is just FastK itself)
    df['fastd'] = df['fastk']

    # calculate Slow %D (EMA of Fast %D with period d_period)
    df['slowd'] = df['fastd'].ewm(span=d_period, adjust=False).mean()

    # drop unecessary columns
    df.drop(columns=['highest_high', 'lowest_low'], inplace=True)

    # Return the dataframe with stochastic values
    return df


In [7]:
def calculate_cci(df: pd.DataFrame, period: int) -> pd.DataFrame:
    """ A method that calculates commodity channel index.

        Parameters
        ----------
        df: pd.DataFrame (Input dataframe containing OHLC data.)
        period: int (lookback period)

        Returns
        -------
        pd.DataFrame (DataFrame that contains Commodity Channel Index (CCI).)
    """
        
    # calculate the typical price
    df['typical_price'] = (df['high'] + df['low'] + df['close']) / 3

    # calculate the simple moving average (SMA) of the Typical Price
    sma = df['typical_price'].rolling(window=period).mean()

    # calculate the mean deviation manually
    mean_deviation = df['typical_price'].rolling(window=period).apply(
        lambda x: (np.abs(x - x.mean()).mean()), raw=True
    )

    # calculate the CCI
    df[f'cci{period}'] = (df['typical_price'] - sma) / \
        (0.015 * mean_deviation)

    # return the resulted dataframe
    return df

In [8]:
gold_6h = calculate_slowD(gold_6h, k_period=9, d_period=3)
gold_6h = calculate_cci(gold_6h, period=3)
gold_6h = calculate_cci(gold_6h, period=9)
# Store only the required columns from gold_6h
gold_6h_selected = gold_6h.reset_index()[['bucket_6h', 'slowd', 'cci3', 'cci9']].rename(columns={'bucket_6h': 'date_time'})
# Add updated_at column with current datetime for all rows
# Example UTC datetime
utc_dt = datetime.now(pytz.utc)
gold_6h_selected['updated_at'] = utc_dt
# Reorder columns as requested
gold_6h_selected = gold_6h_selected[['date_time', 'updated_at', 'slowd', 'cci3', 'cci9']]
print(gold_6h_selected.tail(10))

                    date_time                       updated_at      slowd  \
622 2025-08-08 06:00:00+00:00 2025-08-12 06:33:10.064079+00:00  64.735355   
623 2025-08-08 12:00:00+00:00 2025-08-12 06:33:10.064079+00:00  56.422320   
624 2025-08-08 18:00:00+00:00 2025-08-12 06:33:10.064079+00:00  67.996106   
625 2025-08-10 18:00:00+00:00 2025-08-12 06:33:10.064079+00:00  64.281202   
626 2025-08-11 00:00:00+00:00 2025-08-12 06:33:10.064079+00:00  43.543040   
627 2025-08-11 06:00:00+00:00 2025-08-12 06:33:10.064079+00:00  26.397370   
628 2025-08-11 12:00:00+00:00 2025-08-12 06:33:10.064079+00:00  25.280967   
629 2025-08-11 18:00:00+00:00 2025-08-12 06:33:10.064079+00:00  18.808233   
630 2025-08-12 00:00:00+00:00 2025-08-12 06:33:10.064079+00:00  19.892463   
631 2025-08-12 06:00:00+00:00 2025-08-12 06:33:10.064079+00:00  12.578228   

           cci3        cci9  
622  -66.251638   61.095919  
623 -100.000000   37.397706  
624  100.000000   60.813329  
625   84.966525   78.366239  
62

In [9]:
# save the dataframe as a CSV file
gold_6h_selected.to_csv('gold_6h_indicators.csv', index=False)

In [10]:
gold_daily = calculate_slowD(gold_daily, k_period=9, d_period=3)
gold_daily = calculate_cci(gold_daily, period=3)
gold_daily = calculate_cci(gold_daily, period=9)
# Store only the required columns from gold_daily
gold_daily_selected = gold_daily.reset_index()[['bucket_daily', 'slowd', 'cci3', 'cci9']].rename(columns={'bucket_daily': 'date_time'})
# Add updated_at column with current datetime for all rows
# Example UTC datetime
utc_dt = datetime.now(pytz.utc)
gold_daily_selected['updated_at'] = utc_dt
# Reorder columns as requested
gold_daily_selected = gold_daily_selected[['date_time', 'updated_at', 'slowd', 'cci3', 'cci9']]
print(gold_daily_selected.tail(10))

                    date_time                       updated_at      slowd  \
173 2025-08-01 00:00:00+00:00 2025-08-12 06:33:10.116880+00:00  88.381564   
174 2025-08-03 00:00:00+00:00 2025-08-12 06:33:10.116880+00:00  93.521158   
175 2025-08-04 00:00:00+00:00 2025-08-12 06:33:10.116880+00:00  95.955912   
176 2025-08-05 00:00:00+00:00 2025-08-12 06:33:10.116880+00:00  97.248109   
177 2025-08-06 00:00:00+00:00 2025-08-12 06:33:10.116880+00:00  96.603782   
178 2025-08-07 00:00:00+00:00 2025-08-12 06:33:10.116880+00:00  97.491797   
179 2025-08-08 00:00:00+00:00 2025-08-12 06:33:10.116880+00:00  97.791881   
180 2025-08-10 00:00:00+00:00 2025-08-12 06:33:10.116880+00:00  92.152908   
181 2025-08-11 00:00:00+00:00 2025-08-12 06:33:10.116880+00:00  72.635263   
182 2025-08-12 00:00:00+00:00 2025-08-12 06:33:10.116880+00:00  38.783100   

           cci3        cci9  
173   57.529433  160.134963  
174   86.348283  111.507410  
175   69.342327   84.434826  
176   88.206628   68.750440  
17

In [11]:
gold_daily_selected.to_csv('gold_daily_indicators.csv', index=False)

In [ ]:
def process_indicators(df, bucket_col, output_csv):
    # Calculate indicators
    df = calculate_slowD(df, k_period=9, d_period=3)
    df = calculate_cci(df, period=3)
    df = calculate_cci(df, period=9)
    # Select and rename columns
    selected = df.reset_index()[[bucket_col, 'slowd', 'cci3', 'cci9']].rename(columns={bucket_col: 'date_time'})
    # Add updated_at in Singapore time
    # sg_dt = datetime.now(pytz.utc).astimezone(pytz.timezone('Asia/Singapore'))
    utc_dt = datetime.now(pytz.utc)
    selected['updated_at'] = utc_dt
    # Reorder columns
    selected = selected[['date_time', 'updated_at', 'slowd', 'cci3', 'cci9']]
    # print(selected.tail(10))
    # Save to CSV
    selected.to_csv(output_csv, index=False)
    return selected

In [13]:
gold_6h_1 = process_indicators(gold_6h, 'bucket_6h', 'gold_6h_indicators.csv')

                    date_time                       updated_at      slowd  \
622 2025-08-08 06:00:00+00:00 2025-08-12 14:33:10.191723+08:00  64.735355   
623 2025-08-08 12:00:00+00:00 2025-08-12 14:33:10.191723+08:00  56.422320   
624 2025-08-08 18:00:00+00:00 2025-08-12 14:33:10.191723+08:00  67.996106   
625 2025-08-10 18:00:00+00:00 2025-08-12 14:33:10.191723+08:00  64.281202   
626 2025-08-11 00:00:00+00:00 2025-08-12 14:33:10.191723+08:00  43.543040   
627 2025-08-11 06:00:00+00:00 2025-08-12 14:33:10.191723+08:00  26.397370   
628 2025-08-11 12:00:00+00:00 2025-08-12 14:33:10.191723+08:00  25.280967   
629 2025-08-11 18:00:00+00:00 2025-08-12 14:33:10.191723+08:00  18.808233   
630 2025-08-12 00:00:00+00:00 2025-08-12 14:33:10.191723+08:00  19.892463   
631 2025-08-12 06:00:00+00:00 2025-08-12 14:33:10.191723+08:00  12.578228   

           cci3        cci9  
622  -66.251638   61.095919  
623 -100.000000   37.397706  
624  100.000000   60.813329  
625   84.966525   78.366239  
62

In [14]:
gold_daily_1 = process_indicators(gold_daily, 'bucket_daily', 'gold_daily_indicators.csv')

                    date_time                       updated_at      slowd  \
173 2025-08-01 00:00:00+00:00 2025-08-12 14:33:10.291245+08:00  88.381564   
174 2025-08-03 00:00:00+00:00 2025-08-12 14:33:10.291245+08:00  93.521158   
175 2025-08-04 00:00:00+00:00 2025-08-12 14:33:10.291245+08:00  95.955912   
176 2025-08-05 00:00:00+00:00 2025-08-12 14:33:10.291245+08:00  97.248109   
177 2025-08-06 00:00:00+00:00 2025-08-12 14:33:10.291245+08:00  96.603782   
178 2025-08-07 00:00:00+00:00 2025-08-12 14:33:10.291245+08:00  97.491797   
179 2025-08-08 00:00:00+00:00 2025-08-12 14:33:10.291245+08:00  97.791881   
180 2025-08-10 00:00:00+00:00 2025-08-12 14:33:10.291245+08:00  92.152908   
181 2025-08-11 00:00:00+00:00 2025-08-12 14:33:10.291245+08:00  72.635263   
182 2025-08-12 00:00:00+00:00 2025-08-12 14:33:10.291245+08:00  38.783100   

           cci3        cci9  
173   57.529433  160.134963  
174   86.348283  111.507410  
175   69.342327   84.434826  
176   88.206628   68.750440  
17